# Sentiment Analysis

In [1]:
# Connecting to Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

In [3]:
import re
import string
import gzip
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as pyplot
from pandas import option_context
from sklearn import preprocessing
from tabulate import tabulate
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.cluster import KMeans
from sklearn.ensemble  import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score,accuracy_score,recall_score,precision_score,classification_report,confusion_matrix
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
# Importing Data
data = pd.read_csv('/content/drive/MyDrive/capstone_project/data_files/nlp_data_health.csv')

In [5]:
# Data Shape of Sampled Data
data.shape

(613279, 20)

In [6]:
# Creating Data Frame
data_1 = pd.DataFrame()
data_1["review"]=data["reviewText"]
data_1["sentiment"]=data["sentiment"]

In [7]:
# Displaying Data Frame
with option_context('display.max_colwidth',200):
    display(data_1.head(5))

,review,sentiment
0,"It's done very well. It's strong and well made. The only one issue I had was a scare I had, I picked the bucket up by the handle and the handle slipped off. I caught it before anything spilled....",Positive
1,"Finally I got the hair all over my body shaved, it's just what I want, my girl friend and boy friend all like it, lol!",Positive
2,This tape does not leave any tape residue on your skin. So much better for long term use on delicate sensitive skin. I love it. I have been using it for 1 year this time and for 2 yrs in 2002. I h...,Positive
3,"I've never waxed myself before, but this works great! Putting on a thin layer, rubbing quickly a few times in the direction of growth, and then rapidly pulling the strip off makes it work perfectl...",Positive
4,"I take a pill about 30 minutes before I want to go to sleep, and it doesn't take long before I am yawning and ready to crawl under the covers. If you have problems sleeping at night, I would encou...",Positive


In [8]:
# Defining Function for Cleaning
def cleaning(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [9]:
# Cleaning of Data
data_1['cleaned_review']=data_1['review'].apply(cleaning)
data_1['cleaned_review']

0         its done very well  its strong and well made  ...
1         finally i got the hair all over my body shaved...
2         this tape does not leave any tape residue on y...
3         ive never waxed myself before but this works g...
4         i take a pill about  minutes before i want to ...
                                ...                        
613274    its not a very effective cleaner compared to o...
613275    i started taking thyroid energy in november   ...
613276    even though the head looks like its made of me...
613277    ive never reviewed anything online before but ...
613278    it definitely helped get rid of water weight b...
Name: cleaned_review, Length: 613279, dtype: object

In [10]:
# Stopwords Treatment
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
# Loading Stopwords
sw = stopwords.words('english')

In [12]:
# Defining Function for Stopwords
def stop_treat(data):
  flst = []
  for i in data:
    lst = []
    for j in i.split(' '):
      if j not in sw:
        lst.append(j)
    flst.append(' '.join(lst))
  return(flst)

In [13]:
# Stopwords Treatment
data_1['cleaned_review']=stop_treat(data_1['cleaned_review'])
data_1['cleaned_review']

0         done well  strong well made  one issue scare p...
1         finally got hair body shaved want girl friend ...
2         tape leave tape residue skin much better long ...
3         ive never waxed works great putting thin layer...
4         take pill  minutes want go sleep doesnt take l...
                                ...                        
613274    effective cleaner compared others ive used  ha...
613275    started taking thyroid energy november   first...
613276    even though head looks like made metal im pret...
613277    ive never reviewed anything online thing bad f...
613278    definitely helped get rid water weight didnt w...
Name: cleaned_review, Length: 613279, dtype: object

In [14]:
# Dropping Review Column
data_1.drop(["review"],axis=1,inplace=True)

In [15]:
# Displaying Data
with option_context('display.max_colwidth',200):
    display(data_1.head(5))

,sentiment,cleaned_review
0,Positive,done well strong well made one issue scare picked bucket handle handle slipped caught anything spilled think could make better get make sure pick bucket
1,Positive,finally got hair body shaved want girl friend boy friend like lol
2,Positive,tape leave tape residue skin much better long term use delicate sensitive skin love using year time yrs tried every kind bomb good deal
3,Positive,ive never waxed works great putting thin layer rubbing quickly times direction growth rapidly pulling strip makes work perfectly still little pain course certainly bearable theres slight learning ...
4,Positive,take pill minutes want go sleep doesnt take long yawning ready crawl covers problems sleeping night would encourage anyone everyone try melatonin work everyone dollars nearly years supply worth t...


In [16]:
# Converting Category to Numerical
labelencoder=preprocessing.LabelEncoder()
data_1["sentiment"]=labelencoder.fit_transform(data_1["sentiment"])

In [17]:
# Converting to Unicode
data_1["cleaned_review"] = data_1["cleaned_review"].astype('U').values

In [18]:
# Splitting Dependent and Independent Data
x=data_1["cleaned_review"]
y=data_1["sentiment"]

In [22]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [20]:
# Splitting Train and Test Data
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=42)

In [21]:
# Length of Train and Test
print(f"length of x_train ={len(x_train)},length of y_train ={len(y_train)}")
print(f"length of x_test ={len(x_test)},length of y_test ={len(y_test)}")

length of x_train =429295,length of y_train =429295
length of x_test =183984,length of y_test =183984


In [23]:
# Vectorizing Data
tvectorizer = TfidfVectorizer(ngram_range=(2,2))

In [24]:
# Data Vectorization
x_train_1 = tvectorizer.fit_transform(x_train)
x_test_1 = tvectorizer.transform(x_test)

In [26]:
# Defining Function for Evaluation
def evaluation_metrics(y_test,y_pred):
    print("--------------------------------------------confusion matrix -----------------------------------------")
    cm=confusion_matrix(y_test,y_pred)
    print(cm)
    print()
    print("--------------------------------------------classification report -----------------------------------------")
    cr=classification_report(y_test,y_pred)
    print(cr)
    acc=accuracy_score(y_test,y_pred)
    rec=recall_score(y_test,y_pred,average="weighted")
    f1=f1_score(y_test,y_pred,average="weighted")
    pre=precision_score(y_test,y_pred,average="weighted")
    print('The Accuracy score is : ',round(acc*100,1),'%')
    print('The Precission score is : ',round(pre*100,1),'%')
    print('The F1_score score is : ',round(f1*100,1),'%')
    print('The Recall score is : ',round(rec*100,1),'%')

In [27]:
# Logistic Regression Model Building
lr=LogisticRegression()
lr.fit(x_train_1, y_train)

LogisticRegression()

In [28]:
# LR Model Prediction
pred_lr = lr.predict(x_test_1)

In [29]:
# Evaluation
evaluation_metrics(y_test,pred_lr)

--------------------------------------------confusion matrix -----------------------------------------
[[ 13878    360  13403]
 [  2284    869  11847]
 [  1756    353 139234]]

--------------------------------------------classification report -----------------------------------------
              precision    recall  f1-score   support

           0       0.77      0.50      0.61     27641
           1       0.55      0.06      0.10     15000
           2       0.85      0.99      0.91    141343

    accuracy                           0.84    183984
   macro avg       0.72      0.52      0.54    183984
weighted avg       0.81      0.84      0.80    183984

The Accuracy score is :  83.7 %
The Precission score is :  81.1 %
The F1_score score is :  80.0 %
The Recall score is :  83.7 %


In [30]:
# Multinomial Model Building
mb = MultinomialNB()
mb.fit(x_train_1, y_train)

MultinomialNB()

In [31]:
# MB Model Prediction
pred_mb = mb.predict(x_test_1)

In [32]:
# Evaluation
evaluation_metrics(y_test,pred_mb)

--------------------------------------------confusion matrix -----------------------------------------
[[   353      0  27288]
 [     3      0  14997]
 [     2      0 141341]]

--------------------------------------------classification report -----------------------------------------
              precision    recall  f1-score   support

           0       0.99      0.01      0.03     27641
           1       0.00      0.00      0.00     15000
           2       0.77      1.00      0.87    141343

    accuracy                           0.77    183984
   macro avg       0.59      0.34      0.30    183984
weighted avg       0.74      0.77      0.67    183984

The Accuracy score is :  77.0 %
The Precission score is :  73.9 %
The F1_score score is :  67.2 %
The Recall score is :  77.0 %


In [33]:
from sklearn.svm import LinearSVC

In [34]:
# SVM Model Building
svm = LinearSVC()
svm.fit(x_train_1, y_train)

LinearSVC()

In [36]:
# SVM Model Prediction
pred_svm = svm.predict(x_test_1)

In [37]:
# Evaluation
evaluation_metrics(y_test,pred_svm)

--------------------------------------------confusion matrix -----------------------------------------
[[ 16319    886  10436]
 [  2974   1627  10399]
 [  2507    848 137988]]

--------------------------------------------classification report -----------------------------------------
              precision    recall  f1-score   support

           0       0.75      0.59      0.66     27641
           1       0.48      0.11      0.18     15000
           2       0.87      0.98      0.92    141343

    accuracy                           0.85    183984
   macro avg       0.70      0.56      0.59    183984
weighted avg       0.82      0.85      0.82    183984

The Accuracy score is :  84.8 %
The Precission score is :  81.9 %
The F1_score score is :  82.0 %
The Recall score is :  84.8 %


In [38]:
# Count Vectorizer
cvectorizer = CountVectorizer(ngram_range=(2,2))

In [40]:
# Vectorizing Data
x_train_2 = cvectorizer.fit_transform(x_train)
x_test_2 = cvectorizer.transform(x_test)

In [41]:
# LR Model Building
lr_1 = LogisticRegression()
lr_1.fit(x_train_2,y_train)

LogisticRegression()

In [42]:
# Prediction
pred_lr_1 = lr_1.predict(x_test_2)

In [43]:
# Evaluation
evaluation_metrics(y_test,pred_lr_1)

--------------------------------------------confusion matrix -----------------------------------------
[[ 15847   1260  10534]
 [  2838   2052  10110]
 [  2735   1522 137086]]

--------------------------------------------classification report -----------------------------------------
              precision    recall  f1-score   support

           0       0.74      0.57      0.65     27641
           1       0.42      0.14      0.21     15000
           2       0.87      0.97      0.92    141343

    accuracy                           0.84    183984
   macro avg       0.68      0.56      0.59    183984
weighted avg       0.81      0.84      0.82    183984

The Accuracy score is :  84.2 %
The Precission score is :  81.3 %
The F1_score score is :  81.8 %
The Recall score is :  84.2 %


In [44]:
# Multinomial NB Model Building
mb_1 = MultinomialNB()
mb_1.fit(x_train_2, y_train)

MultinomialNB()

In [45]:
# Prediction
pred_mb_1 = mb_1.predict(x_test_2)

In [46]:
# Evaluation
evaluation_metrics(y_test,pred_mb_1)

--------------------------------------------confusion matrix -----------------------------------------
[[  8459     71  19111]
 [   872     87  14041]
 [   556     57 140730]]

--------------------------------------------classification report -----------------------------------------
              precision    recall  f1-score   support

           0       0.86      0.31      0.45     27641
           1       0.40      0.01      0.01     15000
           2       0.81      1.00      0.89    141343

    accuracy                           0.81    183984
   macro avg       0.69      0.44      0.45    183984
weighted avg       0.78      0.81      0.75    183984

The Accuracy score is :  81.1 %
The Precission score is :  78.3 %
The F1_score score is :  75.5 %
The Recall score is :  81.1 %


In [47]:
# SVM Model Building
svm_1 = LinearSVC()
svm_1.fit(x_train_2,y_train)

LinearSVC()

In [48]:
# Prediction
pred_svm_1 = svm_1.predict(x_test_2)

In [49]:
# Evaluation
evaluation_metrics(y_test,pred_svm_1)

--------------------------------------------confusion matrix -----------------------------------------
[[ 16075   1782   9784]
 [  3177   2459   9364]
 [  3895   2787 134661]]

--------------------------------------------classification report -----------------------------------------
              precision    recall  f1-score   support

           0       0.69      0.58      0.63     27641
           1       0.35      0.16      0.22     15000
           2       0.88      0.95      0.91    141343

    accuracy                           0.83    183984
   macro avg       0.64      0.57      0.59    183984
weighted avg       0.81      0.83      0.81    183984

The Accuracy score is :  83.3 %
The Precission score is :  80.5 %
The F1_score score is :  81.4 %
The Recall score is :  83.3 %
